# DAML - BSU Skills Bootcamp 2025
## Problem statement:
### Are there identifiable features of students’ citation practices that we can use as insights for our teaching?

## ETL
### Extraction
    - Two undergraduate dissertation part 1 submissions. Excellent from 2023-24 and a upper mid-range from 2024-25.
    - References transformed into JSON using anystyle.io
        - TODO: create function that runs local Ruby script for anystyle.io to allow batch processing of multiple scripts.
## Transform JSON into VOSViewer format
### Lookup paperid as <<PK>> from semanticscholar api  

